---
title: Calculating climate normals of temperature and precipitation
short_title: Climate normals
---

This notebook shows how to calculate monthly climate normals for temperature and precipitation for districts in Sierra Leone based on data from [ERA5-Land](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-monthly-means). Climate normals ([see definition](../glossary.md)) are used to as a baseline to evaluate year-to-year variability for weather and climate. 

We will use [earthkit](https://ecmwf.github.io/earthkit-website/), [Xarray](https://xarray.dev/) and [pandas](https://pandas.pydata.org/) (both included in earthkit) to perform the analyis. 

In [61]:
import earthkit.data
from earthkit.transforms import aggregate

Use [earthkit.data](https://earthkit-data.readthedocs.io) to load a NetCDF file containing monthly temperature and precipitation values since 1990: 

In [62]:
file = "../data/era5-land-monthly-temp-precip-1990-2025-sierra-leone.nc"
data = earthkit.data.from_source("file", file)

This file was downloaded from the "[ERA5-Land monthly averaged data from 1950 to present](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-monthly-means)" dataset. See this tutorial for how to download data from the [Climate Data Store](../getting-data/climate-data-store.md).

We convert the data returned to an [Xarray dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html) and display the contents: 

In [63]:
dataset = data.to_xarray()
dataset

<xarray.Dataset> Size: 4MB
Dimensions:     (valid_time: 428, latitude: 33, longitude: 33)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 3kB 1990-01-01 ... 2025-08-01
  * latitude    (latitude) float64 264B 10.1 10.0 9.9 9.8 ... 7.2 7.1 7.0 6.9
  * longitude   (longitude) float64 264B -13.4 -13.3 -13.2 ... -10.4 -10.3 -10.2
    expver      (valid_time) <U4 7kB dask.array<chunksize=(428,), meta=np.ndarray>
Data variables:
    t2m         (valid_time, latitude, longitude) float32 2MB dask.array<chunksize=(428, 33, 33), meta=np.ndarray>
    tp          (valid_time, latitude, longitude) float32 2MB dask.array<chunksize=(428, 33, 33), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-09-29T21:53 GRIB to CDM+CF via cfgrib-0.9.1...

The dataset is covering Sierra Leone, and contains two data variables: 
- t2m: Temperature 2m above surface
- tp: Total precipitation

The spatial resoulution for this gridded dataset is approximately 9x9 km, and the temporal resolution is monthly values.

We can drop the variables that we don't need in our analysis. We will keep time (valid_time) and space (latitude and longitude) dimensions.  

In [64]:
dataset_clean = dataset.drop_vars(['number', 'expver'])

To calculate climate normals we need to select the 30 years reference period (1991-2020).

In [ ]:
dataset_period = dataset_clean.sel(valid_time=slice('1991-01-01', '2020-12-01'))

This will give us 30 years of montly data. We can check that the "valid_time" dimension has 360 items (30 years x 12 months). 

In [77]:
dataset_period.dims

FrozenMappingWarningOnValuesAccess({'valid_time': 360, 'latitude': 33, 'longitude': 33})

To calculate monthly climate normals for temperature we select the variable subtract 273.15 to convert from kelvin to celcius.

In [65]:
temp = dataset_period['t2m'] - 273.15

Next we calculate the mean value for each month of the year.

In [ ]:
temp_month = temp.groupby('valid_time.month').mean()

We can do the same with precipitation. The precipitation values are in meter and is given as the daily avarage for that month. We multiply each value by 1000 to get millimeters, and the numer of days to get the total precipitation for each month.  

In [78]:
precip = dataset_period['tp'] * 1000 * dataset_period.valid_time.dt.days_in_month

We can now caculate the mean precipitation for each month of the year.

In [ ]:
precip_month = precip.groupby('valid_time.month').mean()

The last step is to caculate the climate normals for each district in Sierra Leone. First we need to load the districts from a GeoJSON file [downloaded from DHIS2 Maps](../org-units/download-maps-app.md). 

In [67]:
district_file = "../data/sierra-leone-districts.geojson"
features = earthkit.data.from_source("file", district_file)

To aggregate the temperature data to the org unit features we use the aggregate package of [earthkit-transforms](https://earthkit-transforms.readthedocs.io). See [this notebook](../aggregation/earthkit-netcdf.ipynb) for more information. 

In [79]:
temp_agg = aggregate.spatial.reduce(temp_month, features, mask_dim="id")

We can convert the result to a dataframe to display the normal themperature for each district (id, "O6uvpzGd5pu" is Bo district) and each month (1-12). 

In [84]:
temp_df = temp_agg.to_dataframe()
# temp_df[temp_df['id']=='O6uvpzGd5pu']
temp_df

t2m
id          month           
O6uvpzGd5pu 1      26.410135
            2      27.315334
            3      27.332474
            4      26.646544
            5      25.598896
...                      ...
at6UHUQatSo 8      24.449606
            9      24.824701
            10     25.390593
            11     26.051811
            12     26.593266

[156 rows x 1 columns]

We can do the same to calculate the normal monthly precipitation for each district:

In [69]:
precip_agg = aggregate.spatial.reduce(precip_month, features, mask_dim="id")
precip_df = precip_agg.to_dataframe()

id          month            
O6uvpzGd5pu 1       23.923516
            2       27.693834
            3       74.202901
            4      141.993402
            5      272.604709
...                       ...
at6UHUQatSo 8      738.891497
            9      510.102890
            10     271.774985
            11     132.949604
            12      44.524906

[156 rows x 1 columns]

See [more examples from Copernicus Climate Change Service (C3S)](https://ecmwf-projects.github.io/copernicus-training-c3s/reanalysis-climatology.html) of how to calculate climate normals. 